<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datacamp.com/tutorial/mistral-7b-tutorial

# DEPENDENCIES

In [2]:
import torch
import tensorboard

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
#! pip install datasets ninja packaging --quiet
! pip install datasets  --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [ ]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [3]:
# set device
device = 'cuda'

In [4]:
torch.__version__

'2.3.0+cu121'

In [5]:
#!apt-get update && apt-get install -y cuda-11-0

In [6]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Mon Jun 10 20:48:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0       

In [7]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

# Data loading


https://huggingface.co/datasets/sakharamg/AviationQA

In [8]:
#! pip install datasets  --quiet

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset Emotion")
dataset = load_dataset("dair-ai/emotion")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

In [12]:
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})


In [13]:
train_question=dataset['text']
train_answer=dataset['label']

In [14]:
!rm -rf /content/summary-qa.txt
filename='/content/summary-qa.txt'

import re

# python object to be appended
for n in range(10000):
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    string = train_answer[n]
    #print(string)
    #result = string.replace("\nRef", "-Ref")
    str_question=train_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\n", "", str(string))
    #print(answer)
    if len(answer)==0:
        answer='Not possible to get or use'
    text='<s>[INST] %s [/INST] %s </s>'%(question,answer)
    #print(text)
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [15]:
train_question[8]

'i have been with petronas for years i feel that petronas has performed well and made a huge profit'

label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [16]:
train_answer[8]

1

In [17]:
text0 = load_dataset("text", data_files="/content/summary-qa.txt", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
#text0['text']

In [19]:
nrec=10000

dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
dataset_final_text=text0[0:nrec]['text']

In [20]:
dataset[84]

{'text': 'i feel very happy and excited since i learned so many things',
 'label': 1}

In [21]:
text0[84]

{'text': '<s>[INST] i feel very happy and excited since i learned so many things [/INST] 1 </s>'}

In [22]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

In [23]:
datasetF

,Question,Answer,text
0,i didnt feel humiliated,0,<s>[INST] i didnt feel humiliated [/INST] 0 </s>
1,i can go from feeling so hopeless to so damned...,0,<s>[INST] i can go from feeling so hopeless to...
2,im grabbing a minute to post i feel greedy wrong,3,<s>[INST] im grabbing a minute to post i feel ...
3,i am ever feeling nostalgic about the fireplac...,2,<s>[INST] i am ever feeling nostalgic about th...
4,i am feeling grouchy,3,<s>[INST] i am feeling grouchy [/INST] 3 </s>
...,...,...,...
9995,i have a desk job and sit on my ass all day lo...,4,<s>[INST] i have a desk job and sit on my ass ...
9996,i feel like such a confused person lately sigh,4,<s>[INST] i feel like such a confused person l...
9997,i feel ive been beaten down by the words of me...,0,<s>[INST] i feel ive been beaten down by the w...
9998,i fully understand the feeling of being beaten...,0,<s>[INST] i fully understand the feeling of be...


In [24]:
datasetF['Question'][84]

'i feel very happy and excited since i learned so many things'

In [25]:
datasetF['Answer'][84]

1

In [26]:
datasetF['text'][84]

'<s>[INST] i feel very happy and excited since i learned so many things [/INST] 1 </s>'

In [27]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [28]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       2.3Gi       9.0Gi       9.0Mi        72Gi        80Gi
Swap:             0B          0B          0B


# Loading the Mistral 7B model

In [29]:
!pip install mistral_inference -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #10 june 2024,

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
#model, tokenizer = setup_chat_format(model, tokenizer)

In [31]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

Inference

Build an inference pipeline with tokenizer and the model.

In [32]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [33]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt0="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

prompt = f"Instruct: Find THE answer for the following  question.\n{prompt0}\nOutput:\n"
#prompt = f"Instruct: Find only the answer for the following  question.\n{prompt0}"

#prompt="What are some interesting sites to visit in the Bay Area?"
#prompt="What is the capital of russia?"
outputs = pipe(prompt0, max_new_tokens=128, temperature=0.9,do_sample=True,top_k=30, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt0)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt0):].strip()}")


Question: What is the capital of russia?

Generated Answer:
Moscow


In [34]:
print(outputs[0]['generated_text'])

What is the capital of russia?
Moscow


In [35]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [36]:
datasetF

,Question,Answer,text
0,i didnt feel humiliated,0,<s>[INST] i didnt feel humiliated [/INST] 0 </s>
1,i can go from feeling so hopeless to so damned...,0,<s>[INST] i can go from feeling so hopeless to...
2,im grabbing a minute to post i feel greedy wrong,3,<s>[INST] im grabbing a minute to post i feel ...
3,i am ever feeling nostalgic about the fireplac...,2,<s>[INST] i am ever feeling nostalgic about th...
4,i am feeling grouchy,3,<s>[INST] i am feeling grouchy [/INST] 3 </s>
...,...,...,...
9995,i have a desk job and sit on my ass all day lo...,4,<s>[INST] i have a desk job and sit on my ass ...
9996,i feel like such a confused person lately sigh,4,<s>[INST] i feel like such a confused person l...
9997,i feel ive been beaten down by the words of me...,0,<s>[INST] i feel ive been beaten down by the w...
9998,i fully understand the feeling of being beaten...,0,<s>[INST] i fully understand the feeling of be...


In [37]:
index=2


prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

prompt = datasetF['text'][index]
print(prompt)

im grabbing a minute to post i feel greedy wrong
3
<s>[INST] im grabbing a minute to post i feel greedy wrong [/INST] 3 </s>


MODEL GENERATION - ZERO SHOT

In [ ]:
index=2

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}"

res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=128, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

In [39]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
im grabbing a minute to post i feel greedy wrong
---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
3

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
im grabbing a minute to post i feel greedy wrong

Answer: It's okay to feel greedy sometimes, but it's important to remember that it's not always the right choice. It's important to consider the impact of your actions on others and to strive to make choices that are fair and ethical.


In [40]:
print(len(datasetF))

10000


In [41]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

# Adding the adopter to the layer

In [42]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# Hyperparmeters

In [43]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="Mistral-7B-v0.1_Emotion", # 06/06/2024 #dataset_AviationQA

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=1,                     # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [44]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['Question', 'Answer', 'text'], dtype='object')


In [45]:
print(datasetF['Answer'][2])

3


# Supervised fine-tuning (SFT)  parameters

In [46]:
!pip install trl -q

In [47]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

max_seq_length = 3072 # max sequence length for model and packing of the dataset

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
#model.to(device)


trainer = SFTTrainer(
    model=original_model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    #device_map={"": device},
    #formatting_func=formatting_func,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

# Model training AND Saving the fine-tuned model

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py
#  1406

# save model
trainer.save_model()

In [49]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

# Model evaluation

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [51]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

In [52]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
peft_model_id = "/content/Mistral-7B-v0.1_Emotion"


# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  load_in_8bit=True # Load the model in 8-bit precision
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)


# load into pipeline
generation_pipeline = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

/content


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [53]:
# Clear the cache to release memory
import gc
gc.collect()
torch.cuda.empty_cache()

In [227]:
index=9106

In [228]:
print(f"Question: {datasetF['Question'][index]} \nAnswer: {datasetF['Answer'][index]}")

Question: i may not feel it i m sure the wisdom that comes with age will help 
Answer: 1


In [229]:
prompt=datasetF['Question'][index]
outputs = generation_pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [230]:
import re

response=outputs[0]['generated_text']
response= re.sub("/INST", "", response)
response= re.sub(r"\[\]", "\nSentiment with label:", response)

print(response)

#print(f"Generated Answer:\n{outputs[0]['generated_text']}")
print()
if datasetF['Answer'][index] == 0:
   print(f'Sentiment-ORIGINAL: Sadness(0)')
if datasetF['Answer'][index] == 1:
   print(f'Sentiment-ORIGINAL: Joy(1)')
if datasetF['Answer'][index] == 2:
   print(f'Sentiment-ORIGINAL: Love(2)')
if datasetF['Answer'][index] == 3:
   print(f'Sentiment-ORIGINAL: Anger(3)')
if datasetF['Answer'][index] == 4:
   print(f'Sentiment-ORIGINAL: Fear(4)')
if datasetF['Answer'][index] == 5:
   print(f'Sentiment-ORIGINAL: Surprise(5)')

i may not feel it i m sure the wisdom that comes with age will help me understand it 
Sentiment with label: 1 

Sentiment-ORIGINAL: Joy(1)
